In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation, Concatenate, AveragePooling2D, Input, Flatten ,MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from keras.applications import VGG16
from keras.applications import DenseNet121
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import time
from numpy.linalg import inv, norm
from keras import layers
from keras import models


np.set_printoptions(suppress=True, precision=3)
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/extracted_files')
save_dir = os.path.join(os.getcwd(), 'saved_models_VGG16validation')


# Ορισμός των διαδραστικών φακέλων
train_dir = '/content/drive/MyDrive/extracted_files/train_images'
#test_dir = '/content/drive/MyDrive/extracted_test_files/covid19-pneumonia-normal-chest-xraypa-dataset/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset/normal'
#test_dir = '/content/drive/MyDrive/extracted_files2/train_images'
test_dir = '/content/drive/MyDrive/extracted_files/test_images'
# Διαβάζουμε το αρχείο csv
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/extracted_files/labels_train.csv')
'''
# Δημιουργήστε μια λίστα με τα ονόματα των αρχείων και τις ετικέτες
file_names = df['file_name'].tolist()
class_ids = df['class_id'].astype(str).tolist()  # Εξασφαλίστε ότι οι ετικέτες είναι σε μορφή string
'''

# Επιβεβαιώστε ότι οι ετικέτες είναι σε μορφή string
df['class_id'] = df['class_id'].astype(str)



Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import errno
import pandas as pd
from keras.applications import VGG16
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Συνάρτηση που ορίζει το ρυθμό μάθησης
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    return lr

# Αντικαταστήστε με τη διαδρομή όπου θέλετε να αποθηκευτεί το αρχείο CSV
csv_file_path = '/content/drive/MyDrive/extracted_files/predictions60.csv'

# Δημιουργία DataFrame με τα ονόματα αρχείων και τις ετικέτες (αν υπάρχουν)
# Το data frame πρέπει να έχει τις στήλες 'filename' και, αν υπάρχουν ετικέτες, 'label'
# Εδώ υποθέτουμε πως δεν υπάρχουν ετικέτες.
df_test = pd.DataFrame({'filename': os.listdir(test_dir)})

# Δημιουργία γεννήτριας δεδομένων για τον φάκελο δοκιμής
test_datagen = ImageDataGenerator(rescale=1./255,samplewise_std_normalization=True)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=test_dir,
    target_size=(224, 224),
    batch_size=42,
    class_mode=None,  # Δεν υπάρχουν ετικέτες για το σύνολο δοκιμής
    shuffle=False  # Αφού θέλουμε να διατηρήσουμε τη σειρά των προβλέψεων
)
# Δημιουργία του μοντέλου VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Πάγωμα των βαρών του VGG16
base_model.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # 3 κλάσεις (πνευμονία, COVID, καθαρός πνεύμονας)

# Σύνθεση του μοντέλου
model.compile(optimizer=Adam(lr=lr_schedule(0)), loss='categorical_crossentropy', metrics=['accuracy'])

#εδώ φορτώνουμε τα βάρη που έσωσε το αρχείο DenseNet.ipynd για να γίνει η αξιολόγηση του μοντέλου
#/content/drive/MyDrive/extracted_files/saved_models_DenseNet121validation/DenseNet121-0030-loss0.346-accuracy0.855-val_accuracy0.820.h5
model.load_weights('/content/drive/MyDrive/extracted_files/saved_models_VGG16validation/DenseNet121-ep0018-loss0.296-accuracy0.874-val_accuracy0.830.h5')#FORTWNW TO KALYTERO ARXEIO KAI OXI MONO

# Προεπεξεργασία και πρόβλεψη των δεδομένων δοκιμής
predictions = model.predict(test_generator, verbose=1)

# Αποθήκευση των προβλέψεων σε ένα αρχείο CSV
df_predictions = pd.DataFrame(predictions, columns=[f'feature_{i}' for i in range(predictions.shape[1])])
df_predictions['file_name'] = test_generator.filenames
df_predictions.to_csv(csv_file_path, index=False)
df_predictions = pd.read_csv(csv_file_path)

# Προσθήκη στήλης 'filename' από το generator
df_predictions['file_name'] = test_generator.filenames

# Προεπεξεργασία των προβλέψεων για να επιλέξετε την κύρια κλάση
df_predictions['class_id'] = np.argmax(predictions, axis=1)

# Διατήρηση μόνο των στηλών 'filename' και 'predicted_class'
df_predictions = df_predictions[['file_name', 'class_id']]

# Αποθήκευση των προβλέψεων σε ένα αρχείο CSV
df_predictions.to_csv(csv_file_path, index=False)



/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1469: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn(


Found 1168 validated image filenames.


28/28 [==============================] - 140s 5s/step


'\npercentage_by_class = df_predictions[\'class_id\'].value_counts(normalize=True) * 100\n\n# Εκτύπωση των αποτελεσμάτων\nprint("Ποσοστό κάθε predicted_class:")\nprint(percentage_by_class)\n# Διαβάστε τα CSV αρχεία\ndf_predictions = pd.read_csv(\'/content/drive/MyDrive/extracted_files2/predictions.csv\')\ndf_labels_train = pd.read_csv(\'/content/drive/MyDrive/extracted_files2/labels_train.csv\')\n\n# Συγχώνευση των δύο DataFrames με βάση τη στήλη \'file_name\'\ndf_merged = pd.merge(df_predictions, df_labels_train, on=\'file_name\')\n\n# Υπολογισμός του ποσοστού συμφωνίας\naccuracy = accuracy_score(df_merged[\'class_id_x\'], df_merged[\'class_id_y\']) * 100\n\n\n# Εκτύπωση του ποσοστού συμφωνίας\nprint(f"Ποσοστό Συμφωνίας: {accuracy:.2f}%")\n'